In [83]:
import numpy as np
import tensorflow as tf

In [84]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S, preprocess_input as pp
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [85]:
npz = np.load('/kaggle/input/plants/public_data.npz', allow_pickle=True)
print(npz.files)

['data', 'labels']


In [86]:
data = npz['data']
labels = npz['labels']

In [87]:
binary_labels = np.array([0 if label == 'healthy' else 1 for label in labels])

In [88]:
def remove_outliers(images, labels):
    outliers = []
    for i, image in enumerate(images):
        if np.sum(data[506] - image) == 0 or np.sum(data[338] - image) == 0:
            outliers.append(i)
    
    return np.delete(images, outliers, axis=0), np.delete(labels, outliers)
    
data, binary_labels = remove_outliers(data, binary_labels)

In [89]:
#normalisation
#normalised = data.astype('float32')/255.0
normalised = pp(data)

In [90]:
#from sklearn.model_selection import train_test_split
#X_train, X_val_test, y_train, y_val_test = train_test_split(normalised, binary_labels, test_size=0.3)
#X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5)

In [91]:
from tensorflow.keras.utils import to_categorical
#y_train_categorical = to_categorical(y_train, num_classes=2)
#y_val_categorical = to_categorical(y_val, num_classes=2)
#y_test_categorical = to_categorical(y_test, num_classes=2)

binary_labels_categorical = to_categorical(binary_labels, num_classes=2)

In [92]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size=32

# Create an instance of ImageDataGenerator with desired augmentations
data_gen = ImageDataGenerator(
    # rotation_range=20,       # Degree range for random rotations
    # width_shift_range=0.2,   # Range (as a fraction of total width) for horizontal shifts
    # height_shift_range=0.2,  # Range (as a fraction of total height) for vertical shifts
    # shear_range=0.1,         # Shear Intensity (Shear angle in counter-clockwise direction)
    # zoom_range=0.1,          # Range for random zoom
    horizontal_flip=True,    # Randomly flip inputs horizontally
    vertical_flip = True,
    fill_mode='nearest'      # Strategy for filling in newly created pixels
)

# Apply the data generator to the training data
train_generator = data_gen.flow(data, binary_labels_categorical, batch_size=batch_size)

In [93]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(binary_labels), y=binary_labels)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"Class weights: {class_weights_dict}")

Class weights: {0: 0.8068365043534343, 1: 1.3147661586967945}


In [94]:
from tensorflow.keras.models import load_model

class modelx:
    def __init__(self, path):
        #self.model = load_model(path)
        self.model = path
        
    def predict(self, X):
        #X_processed = self._preprocess(X)
        predictions = self.model.predict(X)
        predictions = self._postprocess(predictions)
        return predictions
    
    def _preprocess(self, X):
        return pp(X)
    
    def _postprocess(self, predictions):
        #return (predictions > 0.5).astype(int)
        return np.argmax(predictions, axis=-1)

In [95]:
def squeeze_excite_block(input_tensor, ratio=16):
    init = input_tensor
    channel_axis = -1  # Assuming channels-last format
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape(se_shape)(se)

    return Multiply()([init, se])

In [96]:
def build_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Load EfficientNetV2L with pretrained weights
    base_model = EfficientNetV2S(include_top=False, weights=None, input_tensor=inputs)
    base_model.load_weights('/kaggle/input/efficientnet-v2s/efficientnetv2-s_notop.h5')

    # Freeze the base model
    # base_model.trainable = False
    base_model.trainable = True
    for layer in base_model.layers[:400]:
        layer.trainable =  False

    # Add SE blocks and build the model
    x = base_model.output
    x = squeeze_excite_block(x)

    # Add final layers for binary classification
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=output)
    return model

In [97]:
# Assuming input shape of (96, 96, 3)
model = build_model((96, 96, 3))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [98]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set the number of epochs and batch size
epochs = 25
#batch_size = 32

# Define the EarlyStopping and ModelCheckpoint callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/kaggle/working/best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model
history = model.fit(
    train_generator, 
    epochs=epochs, 
    #batch_size=batch_size, 
    #validation_data=(X_val, y_val_categorical), 
    class_weight=class_weights_dict, 
    callbacks=[model_checkpoint],
    verbose=1
)

# Load the best saved model
model.load_weights('/kaggle/working/best_model.h5')

Epoch 1/25


2023-11-12 11:07:00.380150: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


157/157 [==============================] - 37s 40ms/step - loss: 0.5103 - accuracy: 0.7536
Epoch 2/25
157/157 [==============================] - 6s 38ms/step - loss: 0.4190 - accuracy: 0.8253
Epoch 3/25
157/157 [==============================] - 6s 39ms/step - loss: 0.3640 - accuracy: 0.8471
Epoch 4/25
157/157 [==============================] - 6s 39ms/step - loss: 0.3326 - accuracy: 0.8623
Epoch 5/25
157/157 [==============================] - 6s 39ms/step - loss: 0.3107 - accuracy: 0.8685
Epoch 6/25
157/157 [==============================] - 6s 39ms/step - loss: 0.2761 - accuracy: 0.8835
Epoch 7/25
157/157 [==============================] - 6s 38ms/step - loss: 0.2692 - accuracy: 0.8865
Epoch 8/25
157/157 [==============================] - 6s 39ms/step - loss: 0.2508 - accuracy: 0.9001
Epoch 9/25
157/157 [==============================] - 6s 39ms/step - loss: 0.2265 - accuracy: 0.9115
Epoch 10/25
157/157 [==============================] - 6s 38ms/step - loss: 0.2266 - accuracy: 0.9117

In [58]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

24/24 [==============================] - 1s 32ms/step - loss: 0.4205 - accuracy: 0.8788
Test loss: 0.42054206132888794
Test accuracy: 0.878828227519989


In [80]:
# Evaluate the model on the test set
from statistics import mean

loss = []
acc = []
for i in range(50):

    X_n, X_m, y_n, y_m = train_test_split(X_test, y_test_categorical, test_size=0.17)

    #test_loss, test_accuracy = model.evaluate(X_m, y_m, verbose=1)
    tester = modelx(model)
    preds = tester.predict(X_m)
    
    #loss.append(test_loss)
    #acc.append(test_accuracy)
    correct = np.sum(preds == np.argmax(y_m, axis=1))
    total = len(y_m)
    acc.append(correct * 1.0 / total)

#print("Avg. Test loss:", mean(loss))
print("Avg. Test accuracy:", mean(acc))

4/4 [==============================] - 0s 26ms/step
Avg. Test accuracy: 0.87703125


-------------------------------------------

------------------------

In [56]:
# Unfreeze the base_model, to improve accuracy
model.trainable = True

fine_tune_at = 400 

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
    layer.trainable =  False

In [57]:
model.compile(optimizer=Adam(learning_rate=1e-6),  # Use a lower learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [58]:
fine_tune_epochs = 100
total_epochs = fine_tune_epochs + epochs

history_fine = model.fit(
    train_generator,
    epochs=total_epochs,  # Total number of epochs to train for
    initial_epoch=history.epoch[-1],  # Start from the last epoch of the previous training session
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

Epoch 20/180


2023-11-11 17:45:24.211014: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


110/110 [==============================] - 44s 86ms/step - loss: 0.2175 - accuracy: 0.9118 - val_loss: 0.3664 - val_accuracy: 0.8655
Epoch 21/180
110/110 [==============================] - 5s 47ms/step - loss: 0.2189 - accuracy: 0.9083 - val_loss: 0.3684 - val_accuracy: 0.8602
Epoch 22/180
110/110 [==============================] - 5s 48ms/step - loss: 0.2303 - accuracy: 0.9063 - val_loss: 0.3667 - val_accuracy: 0.8628
Epoch 23/180
110/110 [==============================] - 6s 50ms/step - loss: 0.2252 - accuracy: 0.9095 - val_loss: 0.3549 - val_accuracy: 0.8615
Epoch 24/180
110/110 [==============================] - 5s 49ms/step - loss: 0.2098 - accuracy: 0.9146 - val_loss: 0.3669 - val_accuracy: 0.8615
Epoch 25/180
110/110 [==============================] - 5s 49ms/step - loss: 0.2207 - accuracy: 0.9072 - val_loss: 0.3579 - val_accuracy: 0.8589
Epoch 26/180
110/110 [==============================] - 6s 50ms/step - loss: 0.2105 - accuracy: 0.9132 - val_loss: 0.3531 - val_accuracy: 0.86

In [72]:
# Evaluate the model on the test set
X_n, X_m, y_n, y_m = train_test_split(X_test, y_test, test_size=0.17)

test_loss, test_accuracy = model.evaluate(X_m, y_m, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

4/4 [==============================] - 0s 33ms/step - loss: 0.2799 - accuracy: 0.8984
Test loss: 0.2799232304096222
Test accuracy: 0.8984375


In [99]:
model.save('/kaggle/working/effv2s-w0-ft-2o-sm-fd')

In [100]:
!zip -r effv2s_2o_fd.zip /kaggle/working/effv2s-w0-ft-2o-sm-fd

  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/ (stored 0%)
  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/variables/ (stored 0%)
  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/variables/variables.index (deflated 79%)
  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/assets/ (stored 0%)
  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/fingerprint.pb (stored 0%)
  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/saved_model.pb (deflated 92%)
  adding: kaggle/working/effv2s-w0-ft-2o-sm-fd/keras_metadata.pb (deflated 96%)


In [ ]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/eff-wo-ft-fd')